In [1]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import f_oneway

In [2]:
from sklearn.model_selection import StratifiedKFold

In [3]:
from sklearn import set_config; 
set_config(transform_output = "pandas")
#pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 70)

In [4]:
df = pd.read_csv('../input/train.csv')

In [5]:
df

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,pct_PL_enq_L6m_of_L12m,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
0,1,5,4,1,0,0,0.000,0.00,0.200,0.800,...,0.0,0.0,0.000,0.0,1,0,PL,PL,696,P2
1,2,1,0,1,0,0,0.000,0.00,1.000,0.000,...,0.0,0.0,0.000,0.0,0,0,ConsumerLoan,ConsumerLoan,685,P2
2,3,8,0,8,1,0,0.125,0.00,1.000,0.000,...,0.0,0.0,0.000,0.0,1,0,ConsumerLoan,others,693,P2
3,5,3,2,1,0,0,0.000,0.00,0.333,0.667,...,0.0,0.0,0.000,0.0,0,0,AL,AL,753,P1
4,6,6,5,1,0,0,0.000,0.00,0.167,0.833,...,1.0,0.0,0.429,0.0,1,0,ConsumerLoan,PL,668,P3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42059,51332,3,0,3,1,0,0.333,0.00,1.000,0.000,...,0.0,0.0,0.000,0.0,0,0,ConsumerLoan,ConsumerLoan,650,P4
42060,51333,4,2,2,0,1,0.000,0.25,0.500,0.500,...,0.0,0.0,0.000,0.0,0,0,others,others,702,P1
42061,51334,2,1,1,1,1,0.500,0.50,0.500,0.500,...,1.0,0.0,1.000,0.0,0,0,ConsumerLoan,others,661,P3
42062,51335,2,1,1,0,0,0.000,0.00,0.500,0.500,...,0.0,0.0,0.000,0.0,0,0,ConsumerLoan,others,686,P2


In [6]:
target = 'Approved_Flag'
catCols = df.select_dtypes(include=['object']).columns.to_list()
catCols.remove(target)

In [7]:
# Association b/w Target(categorical) and all categorical variables
#Chi-Squared test
for col in catCols:
    chi2,pval,_,_ = chi2_contingency(pd.crosstab(df[col],df['Approved_Flag']))
    print(f'{col}---{pval}')

# Since all categorical values have p value <= 0.05 so are associated with the target so we select all the categorical  features

MARITALSTATUS---3.578180861038862e-233
EDUCATION---2.6942265249737532e-30
GENDER---1.907936100186563e-05
last_prod_enq2---0.0
first_prod_enq2---7.84997610555419e-287


In [8]:
numCols = df.select_dtypes(include=['int64','float64']).columns.to_list()
numCols.remove('PROSPECTID')

In [9]:
#Check for multi-collinearity
cols_to_keep = []
colIdx = 0
vifdf = df[numCols]

for i in range(len(numCols)):
    vifVal = variance_inflation_factor(vifdf,colIdx)
    print(f'{numCols[i]} --- {vifVal}')

    if(vifVal <= 6):
        cols_to_keep.append(numCols[i])
        colIdx +=1
    else:
        vifdf = vifdf.drop([numCols[i]], axis=1)



/home/kzcyanide/miniconda3/envs/venv/lib/python3.12/site-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
/home/kzcyanide/miniconda3/envs/venv/lib/python3.12/site-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Total_TL --- inf
Tot_Closed_TL --- inf
Tot_Active_TL --- 11.320180023967996
Total_TL_opened_L6M --- 8.363698035000336
Tot_TL_closed_L6M --- 6.520647877790928
pct_tl_open_L6M --- 5.149501618212625
pct_tl_closed_L6M --- 2.611111040579735
pct_active_tl --- inf


/home/kzcyanide/miniconda3/envs/venv/lib/python3.12/site-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


pct_closed_tl --- 1788.7926256209232
Total_TL_opened_L12M --- 8.601028256477228
Tot_TL_closed_L12M --- 3.8328007921530785
pct_tl_open_L12M --- 6.0996533816467355
pct_tl_closed_L12M --- 5.5813520096427585
Tot_Missed_Pmnt --- 1.985584353098778


/home/kzcyanide/miniconda3/envs/venv/lib/python3.12/site-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Auto_TL --- inf
CC_TL --- 4.809538302819343
Consumer_TL --- 23.270628983464636
Gold_TL --- 30.595522588100053
Home_TL --- 4.3843464059655854
PL_TL --- 3.064658415523423
Secured_TL --- 2.898639771299252
Unsecured_TL --- 4.377876915347322
Other_TL --- 2.2078535836958433
Age_Oldest_TL --- 4.916914200506864
Age_Newest_TL --- 5.214702030064725
time_since_recent_payment --- 3.3861625024231476
num_times_delinquent --- 7.840583309478997
max_recent_level_of_deliq --- 5.255034641721438


/home/kzcyanide/miniconda3/envs/venv/lib/python3.12/site-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


num_deliq_6mts --- inf
num_deliq_12mts --- 7.380634506427232
num_deliq_6_12mts --- 1.4210050015175733
num_times_30p_dpd --- 8.083255010190323
num_times_60p_dpd --- 1.6241227524040114
num_std --- 7.257811920140003
num_std_6mts --- 15.59624383268298
num_std_12mts --- 1.825857047132431
num_sub --- 1.5080839450032664
num_sub_6mts --- 2.172088834824578
num_sub_12mts --- 2.623397553527229
num_dbt --- 2.2959970812106167
num_dbt_6mts --- 7.360578319196446
num_dbt_12mts --- 2.1602387773102554
num_lss --- 2.8686288267891467
num_lss_6mts --- 6.458218003637272
num_lss_12mts --- 2.8474118865638265
recent_level_of_deliq --- 4.753198156284083
tot_enq --- 16.22735475594825
CC_enq --- 6.424377256363877
CC_enq_L6m --- 8.887080381808687
CC_enq_L12m --- 2.3804746142952653
PL_enq --- 8.609513476514548
PL_enq_L6m --- 13.06755093547673
PL_enq_L12m --- 3.500040056654654
time_since_recent_enq --- 1.9087955874813773
enq_L12m --- 17.006562234161628
enq_L6m --- 10.730485153719197
enq_L3m --- 2.3538497522950275
AG

In [10]:
#Checking association b/w Target(Categorical) and all Numerical features
cols_to_keep_num = []

for col in cols_to_keep:
    a = list(df[col])
    b = list(df[target])

    group_P1 = [value for (value,group) in zip(a,b) if group == 'P1']
    group_P2 = [value for (value,group) in zip(a,b) if group == 'P2']
    group_P3 = [value for (value,group) in zip(a,b) if group == 'P3']
    group_P4 = [value for (value,group) in zip(a,b) if group == 'P4']

    f_statistics, pvalue = f_oneway(group_P1,group_P2,group_P3,group_P4)

    if pvalue <= 0.05:
        cols_to_keep_num.append(col)

In [11]:
print(len(cols_to_keep_num))
print(len(cols_to_keep))

37
39


In [12]:
df[target].value_counts()

Approved_Flag
P2    25452
P3     6440
P4     5264
P1     4908
Name: count, dtype: int64

In [13]:
catCols

['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']

In [14]:
#Final features list after feature selection
features = cols_to_keep_num + catCols
features.append(target)

In [15]:
df = df[features]

In [16]:
for col in catCols:
    print(f'{col} --- {df[col].unique()}')

MARITALSTATUS --- ['Married' 'Single']
EDUCATION --- ['12TH' 'GRADUATE' 'SSC' 'POST-GRADUATE' 'UNDER GRADUATE' 'OTHERS'
 'PROFESSIONAL']
GENDER --- ['M' 'F']
last_prod_enq2 --- ['PL' 'ConsumerLoan' 'AL' 'CC' 'others' 'HL']
first_prod_enq2 --- ['PL' 'ConsumerLoan' 'others' 'AL' 'HL' 'CC']


In [17]:
#Dividing categorical features into Ordinal and Nominal categorical features
#For label and one-hot encoding respectively
catOrdCols = ['EDUCATION']
catNomCols = ['MARITALSTATUS','GENDER','last_prod_enq2','first_prod_enq2']

In [18]:
#Ordinal Encoding of Oridinal features
eduUniq = df['EDUCATION'].unique().tolist()
df['EDUCATION'] = df['EDUCATION'].replace(eduUniq,[2,3,1,4,3,1,3])

/tmp/ipykernel_16331/2220449699.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['EDUCATION'] = df['EDUCATION'].replace(eduUniq,[2,3,1,4,3,1,3])


In [19]:
# One Hot Encoding
dfEncoded = pd.get_dummies(df,columns=catNomCols,dtype='int64')
dfEncoded


,pct_tl_open_L6M,pct_tl_closed_L6M,Tot_TL_closed_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt,CC_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,...,last_prod_enq2_ConsumerLoan,last_prod_enq2_HL,last_prod_enq2_PL,last_prod_enq2_others,first_prod_enq2_AL,first_prod_enq2_CC,first_prod_enq2_ConsumerLoan,first_prod_enq2_HL,first_prod_enq2_PL,first_prod_enq2_others
0,0.000,0.00,0,0.000,0,0,0,4,1,4,...,0,0,1,0,0,0,0,0,1,0
1,0.000,0.00,0,0.000,0,0,0,0,0,1,...,1,0,0,0,0,0,1,0,0,0
2,0.125,0.00,0,0.000,1,0,0,0,2,6,...,1,0,0,0,0,0,0,0,0,1
3,0.000,0.00,0,0.000,0,0,0,0,3,0,...,0,0,0,0,1,0,0,0,0,0
4,0.000,0.00,1,0.167,0,0,0,0,6,0,...,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42059,0.333,0.00,0,0.000,0,0,0,0,0,3,...,1,0,0,0,0,0,1,0,0,0
42060,0.000,0.25,1,0.250,0,0,0,0,2,2,...,0,0,0,1,0,0,0,0,0,1
42061,0.500,0.50,1,0.500,0,0,0,0,0,2,...,1,0,0,0,0,0,0,0,0,1
42062,0.000,0.00,1,0.500,0,0,0,0,0,2,...,1,0,0,0,0,0,0,0,0,1


In [20]:
#Predictive Modelling
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [21]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [22]:
xgb_params = {'learning_rate': 0.05097426246942082,
          'n_estimators': 732,
          'max_depth': 4,
          'subsample': 0.8222044805356123,
          'colsample_bytree': 0.9256598045647526,
          'gamma': 0.6509754269480936,
          'reg_alpha': 5.239764778705016,
          'reg_lambda': 2.6504947277703335,
          'objective': 'multi:softmax',
          'num_class': 4 
          }

cb_params = {'learning_rate': 0.03747840176967439,
             'depth': 6,
             'n_estimators': 900,
             'l2_leaf_reg': 0.9918286788888944,
             'colsample_bylevel': 0.9251958707544501,
             'border_count': 192
             }

In [23]:
models = {'XGB':XGBClassifier(**xgb_params),
          'CB':CatBoostClassifier(**cb_params)}

In [24]:
enc = LabelEncoder()
y = dfEncoded[target]
y = enc.fit_transform(y)
X = dfEncoded.drop(target,axis=1)

In [25]:
fold = range(0,5)
score_types = ['Accuracy']+[f'Precision Class {i}' for i in range(0, 4)] + \
              [f'Recall Class {i}' for i in range(0, 4)] + \
              [f'F1 Score Class {i}' for i in range(0, 4)]

multiindex = pd.MultiIndex.from_product([fold,score_types],names=['Fold','Metric'])

FtreImp = pd.DataFrame(columns = models, 
                            index   = [c for c in X.columns ]
                           ).fillna(0)

accuracyscore = pd.DataFrame(index=range(5),columns=models)

In [ ]:
#Training / cross-validation

cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=42)
OOF_Preds = pd.DataFrame()
Scores = pd.DataFrame(index=multiindex,columns=models)

for fold,(train_idx,valid_idx) in enumerate(cv.split(X,y)):
    Xtrain = X.iloc[train_idx]
    Xvalid = X.iloc[valid_idx]
    ytrain = y[train_idx]
    yvalid = y[valid_idx]
    print(f'fold {fold}')

    for mdl in models.keys():
        model = models[mdl]

        

        model.fit(Xtrain,ytrain)

         # Collating feature importance:-
        try:
            FtreImp[model] += model.feature_importances_
        except: 
            pass

        validPreds = model.predict(Xvalid)
        trainPreds = model.predict(Xtrain)
        precision,recall,f1_score,_ = precision_recall_fscore_support(yvalid,validPreds)
        accuracy = accuracy_score(yvalid,validPreds)
        Scores.loc[fold,'Accuracy'][mdl] = accuracy
        accuracyscore.loc[fold][mdl] = accuracy


        for i in range(0,4):
            Scores.loc[fold,f'Precision Class {i}'][mdl] = precision[i]
            Scores.loc[fold,f'Recall Class {i}'][mdl] = recall[i]
            Scores.loc[fold,f'F1 Score Class {i}'][mdl] = f1_score[i]
        

fold 0
0:	learn: 1.3344989	total: 66.6ms	remaining: 59.9s
1:	learn: 1.2866096	total: 78.8ms	remaining: 35.4s
2:	learn: 1.2430310	total: 96.4ms	remaining: 28.8s
3:	learn: 1.2043908	total: 112ms	remaining: 25s
4:	learn: 1.1694826	total: 131ms	remaining: 23.4s
5:	learn: 1.1367997	total: 148ms	remaining: 22.1s
6:	learn: 1.1067338	total: 165ms	remaining: 21s
7:	learn: 1.0791502	total: 181ms	remaining: 20.2s
8:	learn: 1.0538672	total: 198ms	remaining: 19.6s
9:	learn: 1.0294935	total: 214ms	remaining: 19s
10:	learn: 1.0062547	total: 233ms	remaining: 18.8s
11:	learn: 0.9865644	total: 248ms	remaining: 18.4s
12:	learn: 0.9670275	total: 262ms	remaining: 17.9s
13:	learn: 0.9476038	total: 280ms	remaining: 17.7s
14:	learn: 0.9309632	total: 297ms	remaining: 17.5s
15:	learn: 0.9143049	total: 314ms	remaining: 17.4s
16:	learn: 0.8985738	total: 329ms	remaining: 17.1s
17:	learn: 0.8840524	total: 346ms	remaining: 16.9s
18:	learn: 0.8701343	total: 359ms	remaining: 16.7s
19:	learn: 0.8571941	total: 374ms	rem

In [27]:
Scores

XGB        CB
Fold Metric                               
0    Accuracy            0.78034  0.781885
     Precision Class 0  0.827843  0.830087
     Precision Class 1  0.824552  0.826791
     Precision Class 2  0.452107  0.458182
     Precision Class 3  0.740704  0.748728
     Recall Class 0     0.794088  0.781855
     Recall Class 1     0.922216  0.922609
     Recall Class 2     0.274845  0.293478
     Recall Class 3     0.699905  0.698955
     F1 Score Class 0   0.810614  0.805249
     F1 Score Class 1   0.870654  0.872076
     F1 Score Class 2   0.341864  0.357785
     F1 Score Class 3   0.719727  0.722986
1    Accuracy           0.782361   0.78664
     Precision Class 0  0.803109   0.81164
     Precision Class 1  0.827953  0.830356
     Precision Class 2  0.471769  0.483948
     Precision Class 3  0.753208  0.770021
     Recall Class 0      0.79001  0.781855
     Recall Class 1     0.916912  0.922019
     Recall Class 2     0.291925  0.315994
     Recall Class 3     0.724596  0.712251
     F1 Score Class 0   0.796506  0.796469
     F1 Score Class 1   0.870165   0.87379
     F1 Score Class 2   0.360671  0.382339
     F1 Score Class 3   0.738625   0.74001
2    Accuracy           0.784381  0.789017
     Precision Class 0  0.813929  0.829968
     Precision Class 1  0.826156  0.826933
     Precision Class 2  0.472585  0.486911
     Precision Class 3  0.757028  0.765174
     Recall Class 0     0.797352  0.795316
     Recall Class 1     0.923379  0.926523
     Recall Class 2     0.281056   0.28882
     Recall Class 3     0.716049  0.730294
     F1 Score Class 0   0.805556  0.812272
     F1 Score Class 1   0.872066    0.8739
     F1 Score Class 2   0.352483  0.362573
     F1 Score Class 3   0.735969  0.747328
3    Accuracy            0.78034  0.784381
     Precision Class 0  0.804233  0.819742
     Precision Class 1  0.823116  0.825667
     Precision Class 2  0.473755  0.473684
     Precision Class 3  0.743217  0.753171
     Recall Class 0     0.773931  0.778004
     Recall Class 1     0.920629  0.923969
     Recall Class 2     0.273292  0.279503
     Recall Class 3     0.728395  0.733143
     F1 Score Class 0   0.788791  0.798328
     F1 Score Class 1   0.869146  0.872056
     F1 Score Class 2   0.346627  0.351562
     F1 Score Class 3   0.735731  0.743022
4    Accuracy           0.783642  0.786377
     Precision Class 0   0.81888  0.831818
     Precision Class 1  0.821935  0.823765
     Precision Class 2   0.47027  0.473885
     Precision Class 3  0.765326  0.777559
     Recall Class 0     0.759674  0.745418
     Recall Class 1     0.923183  0.927505
     Recall Class 2     0.270186   0.28882
     Recall Class 3     0.759506  0.750951
     F1 Score Class 0   0.788167  0.786251
     F1 Score Class 1   0.869622  0.872563
     F1 Score Class 2   0.343195    0.3589
     F1 Score Class 3   0.762405  0.764023

In [47]:

accuracyscore

,XGB,CB
0,0.78034,0.781885
1,0.782361,0.78664
2,0.784381,0.789017
3,0.78034,0.784381
4,0.783642,0.786377


In [43]:
accuracyscore.mean().to_dict()

{'XGB': 0.7822128525476192, 'CB': 0.7856599637847343}

In [46]:
max(accuracyscore.mean().to_dict(), key=lambda k: accuracyscore.mean().to_dict()[k])

'CB'

In [30]:
accuracyscore['CB'].mean()

0.7856599637847343

In [44]:
my_dict = {'a': 10, 'b': 20, 'c': 30}

max_key = max(my_dict, key=lambda k: my_dict[k])

print("Maximum element key:", max_key)


Maximum element key: c


In [31]:
#Hyper-Parameter Tuning
import optuna

/home/kzcyanide/miniconda3/envs/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [32]:
def objective(trial):

    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 10.0),
        'objective': 'multi:softmax',  # For multiclass classification
        #'eval_metric': 'mlogloss',      # Evaluation metric
        'num_class': 4       # Number of classes in your dataset
    }

    cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=42)
    scores = []

    for fold,(train_idx,valid_idx) in enumerate(cv.split(X,y)):
        Xtrain = X.iloc[train_idx]
        Xvalid = X.iloc[valid_idx]
        ytrain = y[train_idx]
        yvalid = y[valid_idx]
        #print(f'fold {fold}')

        model = XGBClassifier(**params)

        model.fit(Xtrain,ytrain)

        validPreds = model.predict(Xvalid)
        accuracy = accuracy_score(yvalid,validPreds)
        scores.append(accuracy)
    return np.mean(scores)



In [33]:
study = optuna.create_study(direction='maximize', study_name="optuna_xgboost")
study.optimize(objective, n_trials=5)
print(f"Best trial average accuracy: {study.best_value:.4f}")
for key, value in study.best_params.items():
    print(f"{key}: {value}")

[I 2024-04-29 12:29:49,038] A new study created in memory with name: optuna_xgboost
[I 2024-04-29 12:31:06,110] Trial 0 finished with value: 0.7793363603720189 and parameters: {'learning_rate': 0.11626654574965782, 'n_estimators': 56, 'max_depth': 7, 'subsample': 0.7741528960024443, 'colsample_bytree': 0.8443602416039097, 'gamma': 0.6253418833410195, 'reg_alpha': 2.5550640780484732, 'reg_lambda': 4.265106235532603}. Best is trial 0 with value: 0.7793363603720189.
[W 2024-04-29 12:39:42,707] Trial 1 failed with parameters: {'learning_rate': 0.01976253259154641, 'n_estimators': 748, 'max_depth': 5, 'subsample': 0.8268956751676262, 'colsample_bytree': 0.7183256718214669, 'gamma': 0.9840843209560928, 'reg_alpha': 4.67032643169206, 'reg_lambda': 2.187499617636066} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/kzcyanide/miniconda3/envs/venv/lib/python3.12/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_val

KeyboardInterrupt: 

In [ ]:
def objective(trial):
    
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'depth': trial.suggest_int('depth', 4, 10),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=100),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10.0),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.1, 1.0),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'border_count': trial.suggest_int('border_count', 32, 255, step=32)
    }

    cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=42)
    scores = []

    for fold,(train_idx,valid_idx) in enumerate(cv.split(X,y)):
        Xtrain = X.iloc[train_idx]
        Xvalid = X.iloc[valid_idx]
        ytrain = y[train_idx]
        yvalid = y[valid_idx]
        #print(f'fold {fold}')

        model = CatBoostClassifier(**params)

        model.fit(Xtrain,ytrain)

        validPreds = model.predict(Xvalid)
        accuracy = accuracy_score(yvalid,validPreds)
        scores.append(accuracy)
    return np.mean(scores)

In [ ]:
study = optuna.create_study(direction='maximize', study_name="optuna_catboost")
study.optimize(objective, n_trials=5)
print(f"Best trial average accuracy: {study.best_value:.4f}")
for key, value in study.best_params.items():
    print(f"{key}: {value}")

In [ ]:
model = XGBClassifier(**xgb_params)
Xtrain,Xtest,ytrain,ytest = train_test_split(X,y,test_size=0.2,random_state=42)
model.fit(Xtrain,ytrain)
print(model.feature_importances_)

In [ ]:
# plot feature importance
from xgboost import plot_importance
plot_importance(model)
plt.tight_layout()
plt.show()